# Imports

In [0]:
!pip install category_encoders

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate

import pandas as pd

import category_encoders as ce

from collections import Counter
import time
import string

# Data

In [0]:
# Train data
!wget -O ./train.csv --no-check-certificate "https://docs.google.com/uc?export=download&id=113cuCNN0cPPxvB2003MaZ5cmSL8O4VIK"

--2020-03-11 03:39:59--  https://docs.google.com/uc?export=download&id=113cuCNN0cPPxvB2003MaZ5cmSL8O4VIK
Resolving docs.google.com (docs.google.com)... 172.217.214.138, 172.217.214.100, 172.217.214.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.214.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hg9r5le9scjjvjusi2s504kgbb8svdml/1583898000000/09171232683117075090/*/113cuCNN0cPPxvB2003MaZ5cmSL8O4VIK?e=download [following]
--2020-03-11 03:40:01--  https://doc-08-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hg9r5le9scjjvjusi2s504kgbb8svdml/1583898000000/09171232683117075090/*/113cuCNN0cPPxvB2003MaZ5cmSL8O4VIK?e=download
Resolving doc-08-ag-docs.googleusercontent.com (doc-08-ag-docs.googleusercontent.com)... 108.177.111.132, 2607:f8b0:4001:c07::84
Connecting to doc-08-ag-docs.googleusercontent.com (doc-08

In [0]:
# Test data
!wget -O ./test.csv --no-check-certificate "https://docs.google.com/uc?export=download&id=1aKVGUejm5PNBroaJg2r9up6SGyy58bJb"

--2020-03-11 03:40:03--  https://docs.google.com/uc?export=download&id=1aKVGUejm5PNBroaJg2r9up6SGyy58bJb
Resolving docs.google.com (docs.google.com)... 172.217.214.113, 172.217.214.101, 172.217.214.100, ...
Connecting to docs.google.com (docs.google.com)|172.217.214.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ga7aaktuh99b8jto2rk3ugabg6qn4eur/1583898000000/09171232683117075090/*/1aKVGUejm5PNBroaJg2r9up6SGyy58bJb?e=download [following]
--2020-03-11 03:40:04--  https://doc-10-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ga7aaktuh99b8jto2rk3ugabg6qn4eur/1583898000000/09171232683117075090/*/1aKVGUejm5PNBroaJg2r9up6SGyy58bJb?e=download
Resolving doc-10-ag-docs.googleusercontent.com (doc-10-ag-docs.googleusercontent.com)... 108.177.111.132, 2607:f8b0:4001:c07::84
Connecting to doc-10-ag-docs.googleusercontent.com (doc-10

In [0]:
original_train=pd.read_csv('train.csv',delimiter=',') 
original_train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [0]:
original_test=pd.read_csv('test.csv',delimiter=',') 
original_test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,0870b0a5d,9ceb19dd6,530f8ecc3,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
1,300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,a5c276589,1ad744242,12e6161c9,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
2,300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,568550f04,1fe17a1fd,27d6df03f,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
3,300003,0,0,1,T,Y,Red,Star,Cat,China,Piano,c5725677e,a6542cec0,30c63bd0c,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
4,300004,0,1,1,F,N,Red,Trapezoid,Dog,China,Piano,e70a6270d,97b6a3518,a42386065,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


The data essentially looks like this:

|    | Name   | dtypes | Missing | Uniques |
|----|--------|--------|---------|---------|
| 0  | id     | int64  | 0       | 300000  |
| 1  | bin_0  | int64  | 0       | 2       |
| 2  | bin_1  | int64  | 0       | 2       |
| 3  | bin_2  | int64  | 0       | 2       |
| 4  | bin_3  | object | 0       | 2       |
| 5  | bin_4  | object | 0       | 2       |
| 6  | nom_0  | object | 0       | 3       |
| 7  | nom_1  | object | 0       | 6       |
| 8  | nom_2  | object | 0       | 6       |
| 9  | nom_3  | object | 0       | 6       |
| 10 | nom_4  | object | 0       | 4       |
| 11 | nom_5  | object | 0       | 222     |
| 12 | nom_6  | object | 0       | 522     |
| 13 | nom_7  | object | 0       | 1220    |
| 14 | nom_8  | object | 0       | 2215    |
| 15 | nom_9  | object | 0       | 11981   |
| 16 | ord_0  | int64  | 0       | 3       |
| 17 | ord_1  | object | 0       | 5       |
| 18 | ord_2  | object | 0       | 6       |
| 19 | ord_3  | object | 0       | 15      |
| 20 | ord_4  | object | 0       | 26      |
| 21 | ord_5  | object | 0       | 192     |
| 22 | day    | int64  | 0       | 7       |
| 23 | month  | int64  | 0       | 12      |
| 24 | target | int64  | 0       | 2       |

In [0]:
complete_data = original_train.append(original_test, sort=False)
num_train = len(original_train)
complete_data.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0.0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0.0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0.0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1.0
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0.0


In [0]:
train = complete_data.drop(columns="target")
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 199999
Data columns (total 24 columns):
id       500000 non-null int64
bin_0    500000 non-null int64
bin_1    500000 non-null int64
bin_2    500000 non-null int64
bin_3    500000 non-null object
bin_4    500000 non-null object
nom_0    500000 non-null object
nom_1    500000 non-null object
nom_2    500000 non-null object
nom_3    500000 non-null object
nom_4    500000 non-null object
nom_5    500000 non-null object
nom_6    500000 non-null object
nom_7    500000 non-null object
nom_8    500000 non-null object
nom_9    500000 non-null object
ord_0    500000 non-null int64
ord_1    500000 non-null object
ord_2    500000 non-null object
ord_3    500000 non-null object
ord_4    500000 non-null object
ord_5    500000 non-null object
day      500000 non-null int64
month    500000 non-null int64
dtypes: int64(7), object(17)
memory usage: 95.4+ MB


# Binary Features

bin_0, bin_1 and bin_2 have dtypes of int64 so we don't do anything about them. However, we have to encode bin_3 and bin_4 by simply mapping them to 0's and 1's.

In [0]:
bin3_map = {'T': 1, 'F': 0}

bin4_map = {'Y': 1, 'N': 0}

train['bin_3'] = train['bin_3'].map(bin3_map)
train['bin_4'] = train['bin_4'].map(bin4_map)

# Ordinal Features (with <100 uniques)

ord_0 has numerical values, so we skip it. ord_1, ord_2, ord_3 and ord_4 have significantly fewer uniques than ord_5, so we simply map them to numerical values. ord_5 is handled separately in the next section.

In [0]:
ord1_map = {'Novice': 1, 'Contributor': 2,
               'Expert': 3, 'Master': 4, 'Grandmaster': 5}

ord2_map = {'Freezing': 1, 'Cold': 2,
               'Warm': 3, 'Hot': 4, 'Boiling Hot': 5, 'Lava Hot': 6}

ord3_map = {'a': 1, 'b': 2,
               'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8,
               'i': 9, 'j': 10, 'k': 11, 'l': 12,'m': 13, 'n': 14, 'o': 15}

ord4_map = {'A': 1, 'B': 2,
               'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8,
               'I': 9, 'J': 10, 'K': 11, 'L': 12,'M': 13, 'N': 14, 'O': 15,'P': 16, 'Q': 17,
               'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23,
               'X': 24, 'Y': 25, 'Z': 26}


train['ord_1'] = train['ord_1'].map(ord1_map)
train['ord_2'] = train['ord_2'].map(ord2_map)
train['ord_3'] = train['ord_3'].map(ord3_map)
train['ord_4'] = train['ord_4'].map(ord4_map)

# Ordinal Features (with >100 uniques)



In [0]:
def getASCII(letter):
    return string.ascii_letters.find(letter) + 1

In [0]:
train['ord_5_left'] = train['ord_5'].apply(lambda x: getASCII(x[0]))
train['ord_5_right'] = train['ord_5'].apply(lambda x: getASCII(x[1]))

train = train.drop(columns="ord_5")
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,day,month,ord_5_left,ord_5_right
0,0,0,0,0,1,1,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,5,2,8,4,2,2,11,18
1,1,0,1,0,1,1,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,5,4,1,1,7,8,2,32
2,2,0,0,0,0,1,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,3,6,8,18,7,2,36,3
3,3,0,1,0,0,1,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,5,5,9,4,2,1,11,49
4,4,0,0,0,0,0,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,5,1,1,18,7,8,17,42


# Cyclical Features

day and month are cyclical in nature, so we can do the following:

In [0]:
train['dy_sin'] = np.sin((train['day']-1)*(2.*np.pi/7))
train['dy_cos'] = np.cos((train['day']-1)*(2.*np.pi/7))
train['mnth_sin'] = np.sin((train['month']-1)*(2.*np.pi/12))
train['mnth_cos'] = np.cos((train['month']-1)*(2.*np.pi/12))

train = train.drop(columns="day")
train = train.drop(columns="month")
train = train.drop(columns="id")
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5_left,ord_5_right,dy_sin,dy_cos,mnth_sin,mnth_cos
0,0,0,0,1,1,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,5,2,8,4,11,18,0.781831,0.62349,0.5,0.866025
1,0,1,0,1,1,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,5,4,1,1,2,32,-0.781831,0.62349,-0.5,-0.866025
2,0,0,0,0,1,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,3,6,8,18,36,3,-0.781831,0.62349,0.5,0.866025
3,0,1,0,0,1,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,5,5,9,4,11,49,0.781831,0.62349,0.0,1.000000
4,0,0,0,0,0,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,5,1,1,18,17,42,-0.781831,0.62349,-0.5,-0.866025


# Nominal Features (Low Cardinality)

In [0]:
column_trans = make_column_transformer((OneHotEncoder(sparse=False),['nom_0','nom_1','nom_2','nom_3','nom_4']),remainder='passthrough')
train_after_low_car_nom = column_trans.fit_transform(train)

In [0]:
pd.DataFrame(train_after_low_car_nom).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,5,2,8,4,11,18,0.781831,0.62349,0.5,0.866025
1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,5,4,1,1,2,32,-0.781831,0.62349,-0.5,-0.866025
2,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,3,6,8,18,36,3,-0.781831,0.62349,0.5,0.866025
3,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,5,5,9,4,11,49,0.781831,0.62349,0,1
4,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,5,1,1,18,17,42,-0.781831,0.62349,-0.5,-0.866025


# Nominal Features (High Cardinality)

In [0]:
hashing_encoder = ce.HashingEncoder(cols=[30, 31, 32, 33, 34])
train_after_high_car_nom = hashing_encoder.fit_transform(train_after_low_car_nom)

In [0]:
pd.DataFrame(train_after_high_car_nom).head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,35,36,37,38,39,40,41,42,43,44,45
0,0,0,1,0,1,1,0,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,1,1,2,5,2,8,4,11,18,0.781831,0.62349,0.5,0.866025
1,0,2,2,0,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0,1,0,1,1,1,5,4,1,1,2,32,-0.781831,0.62349,-0.5,-0.866025
2,1,1,0,2,0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,0,0,0,1,1,3,6,8,18,36,3,-0.781831,0.62349,0.5,0.866025
3,0,1,0,0,0,1,1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1,0,0,1,1,5,5,9,4,11,49,0.781831,0.62349,0.0,1.000000
4,1,0,2,1,0,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,0,1,5,1,1,18,17,42,-0.781831,0.62349,-0.5,-0.866025


# Encoding Results

In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 49 columns):
col_0    500000 non-null int64
col_1    500000 non-null int64
col_2    500000 non-null int64
col_3    500000 non-null int64
col_4    500000 non-null int64
col_5    500000 non-null int64
col_6    500000 non-null int64
col_7    500000 non-null int64
0        500000 non-null float64
1        500000 non-null float64
2        500000 non-null float64
3        500000 non-null float64
4        500000 non-null float64
5        500000 non-null float64
6        500000 non-null float64
7        500000 non-null float64
8        500000 non-null float64
9        500000 non-null float64
10       500000 non-null float64
11       500000 non-null float64
12       500000 non-null float64
13       500000 non-null float64
14       500000 non-null float64
15       500000 non-null float64
16       500000 non-null float64
17       500000 non-null float64
18       500000 non-null float64
19       50000

In [0]:
X_train = train[:num_train]
y_train = original_train['target'].values

X_test = train[num_train:]

In [0]:
print("X_train: ", X_train.shape[0])
print("y_train: ", y_train.shape[0])
print("X_test: ", X_test.shape[0])

X_train:  300000
y_train:  300000
X_test:  200000


# Make Prediction

## Logistic Regression

In [0]:
clf=LogisticRegression(C=0.123456789, solver="lbfgs", max_iter=5000)  # MODEL

clf.fit(X_train, y_train)

pred=clf.predict_proba(X_test)[:,1]

pd.DataFrame({"id": original_test["id"], "target": pred}).to_csv("logistic_regression_submission.csv", index=False)

In [0]:
score=cross_validate(clf, X_train, y_train, cv=3, scoring="roc_auc")["test_score"].mean()
print(f"{score:.6f}")

0.747701


## AdaBoost Classifier

In [0]:
adaClf = AdaBoostClassifier()

adaClf.fit(X_train, y_train)

pred = adaClf.predict_proba(X_test)[:,1]

pd.DataFrame({"id": original_test["id"], "target": pred}).to_csv("adaboost_submission.csv", index=False)

In [0]:
score=cross_validate(adaClf, X_train, y_train, cv=3, scoring="roc_auc")["test_score"].mean()
print(f"{score:.6f}")

0.761045
